In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
 # Install colab_ssh on google colab
 !pip install colab_ssh --upgrade
	
 from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
 launch_ssh_cloudflared(password="tjrals")
	
 # Optional: if you want to clone a github repository
 # init_git_cloudflared(githubRepositoryUrl)

In [4]:
import numpy as np; np.random.seed(1234)
import pandas as pd


ntrain = 150000

data = pd.read_csv('/content/drive/MyDrive/sentimental_analisis/ratings.txt', sep='\t', quoting=3)
data = pd.DataFrame(np.random.permutation(data))
trn, tst = data[:ntrain], data[ntrain:]

header = 'id document label'.split()
trn.to_csv('/content/drive/MyDrive/sentimental_analisis/ratings_train.txt', sep='\t', index=False, header=header)
tst.to_csv('/content/drive/MyDrive/sentimental_analisis/ratings_test.txt', sep='\t', index=False, header=header)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/sentimental_analisis/ratings.txt', sep='\t')

In [7]:
# !pip install transformers

In [8]:
import transformers
from transformers import AutoTokenizer

In [ ]:
!python /content/drive/MyDrive/sentimental_analisis/good-news-sentimental-analysis/train.py

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch[3/10](5620/14999) || training loss 0.1003 || training accuracy 95.62% || lr 5.317974446451267e-06
Epoch[3/10](5640/14999) || training loss 0.2196 || training accuracy 89.38% || lr 5.3169041407452e-06
Epoch[3/10](5660/14999) || training loss 0.1221 || training accuracy 94.38% || lr 5.315833835039133e-06
Epoch[3/10](5680/14999) || training loss 0.1446 || training accuracy 95.62% || lr 5.314763529333065e-06
Epoch[3/10](5700/14999) || training loss 0.1669 || training accuracy 94.38% || lr 5.3136932236269985e-06
Epoch[3/10](5720/14999) || training loss 0.1472 || training accuracy 93.75% || lr 5.312622917920931e-06
Epoch[3/10](5740/14999) || training loss 0.1431 || training accuracy 93.75% || lr 5.311552612214863e-06
Epoch[3/10](5760/14999) || training loss 0.1804 || training accuracy 92.50% || lr 5.310482306508796e-06
Epoch[3/10](5780/14999) || training loss 0.1197 || training accuracy 95.00% || lr 5.30941200080273e-06
Epoch[3/10](5800/14999) || tra

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertPreTrainedModel, ElectraModel, ElectraPreTrainedModel, XLMRobertaModel, RobertaModel, BartModel, BartPretrainedModel, PreTrainedModel
import transformers
from transformers import AutoTokenizer

class FCLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.0, use_activation=True):
        super(FCLayer, self).__init__()
        self.use_activation = use_activation
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, output_dim)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.dropout(x)
        if self.use_activation:
            x = self.tanh(x)
        return self.linear(x)

class xlmRoBerta(XLMRobertaModel):
    def __init__(self, config, args):
        super(xlmRoBerta, self).__init__(config)
        self.xlmroberta = XLMRobertaModel.from_pretrained("xlm-roberta-large", config=config)  # Load pretrained Electra

        self.num_labels = config.num_labels

        self.cls_fc_layer = FCLayer(config.hidden_size, config.hidden_size, args.dropout_rate)
        # l2 norm, similarity add
        self.label_classifier = FCLayer(
            config.hidden_size,
            config.num_labels,
            args.dropout_rate,
            use_activation=False,
        )

    def forward(self, input_ids, attention_mask, token_type_ids, labels):
        outputs = self.xlmroberta(
            input_ids, attention_mask=attention_mask
        )  # sequence_output, pooled_output, (hidden_states), (attentions)
        sequence_output = outputs[0]
        pooled_output = outputs[0][:, 0, :]  # [CLS]

        # Average
        pooled_output = self.cls_fc_layer(pooled_output)

        # Concat -> fc_layer
        logits = self.label_classifier(pooled_output)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        # Softmax
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
import easydict

In [ ]:
args = easydict.EasyDict({ "dropout_rate": 0 })

In [ ]:
model = xlmRoBerta.from_pretrained('/content/drive/MyDrive/sentimental_analisis/good-news-sentimental-analysis/results/xlm_batch8_kfold_1/1/best', args=args)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

xlmRoBerta(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dr

In [ ]:
model.eval()

xlmRoBerta(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dr

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")

In [ ]:
qurry = ['이런 기사 쓸시간 있으면 잠이나 자라.',
         '마음이 따뜻해지는 기사네요.',
         '어휴 진짜 아침부터 이런 글 읽으니깐 기분 안좋네요',
         '사랑이 싹트는 기사네요',
         '와 앞으로 어떻게 진행될지 지켜봐야겠어요',
         '뻔하네요',
         'fun 하네요',
         '펀하네요'
         ]

In [ ]:
qurry = ['와 앞으로 어떻게 진행될지 지켜봐야겠어요',
         '뻔하네요',
         'fun 하네요',
         '펀하네요',
         '내가 이럴 줄 알았다']

In [ ]:
qurry = ['살면서 저런 돌I들 안만나게해주세요...',
         '그 손주란 애가 불쌍하네요.',
         '저런할미랑 애미밑에서 뭘 보고 배울지..',
         '애미도 보니 지엄만지 시몬지 몰라도 보고 배운게 딱 수준이 같네요...',
         '목사는 길거리의 똥강아지도 하는 것인줄 이제야 바로 알았다 ',
         '개나 소나 다 목사 하는게 우리나라구나 ',
         '우리나라로 돌아와 값 싼 노동만 원하다가는 큰 코 다쳐',
         '베트남이 이렇게 뒷통수를 칠줄이야.. 정말 몰랐네']

In [ ]:
qurry = ['사장님의 마음과 "사장님처럼 어려운 사람을 도와주는 멋있는 사람이 되겠다”는 다짐과 함께 그동안 동생을 챙겨준 사장님께 감사의 인사를 전한 형의 행동이 너무 아름답네요.저도 이 어려운 시국에 이웃을 도와주며 살아가야겠습니다.마음이 따뜻해지는 기사였습니다.',
         '치킨을 너무 먹고 싶어하는 초등학생 동생을 위해 오천어치만 시켜야했던 형의 가슴아픈 사연에 마음이 너무 아팠어요ㅠㅠ 하지만 사장님이 배불리 치킨을 먹게해주시고 그런 따뜻한 마음이 너무 감사했던 형의 편지로 온세상에 알려지게 됐네요.']

In [ ]:
qurry = ['계속 퇴보하다가 어디까지 가게되는 걸까?']

In [ ]:
qurry = ['무섭기만 하다']

In [ ]:
qurry = ['정부합동특별수사본부(합수본)는 2일 정부서울청사에서 수사 중간 결과 브리핑을 열고 지난 3월 10일 출범부터 이날까지 총 646건의 사건과 관련해, 2796명을 내·수사해 20명을 구속했다고 밝혔다. 또 529명을 검찰에 송치했다. 경찰 중심의 합수본은 1560여명의 규모로 구성됐다. 검찰에도 600여 명의 전담수사팀을 편성했다. 또 금융위원회, 금융감독원, 국세청은 부동산 시장에서의 편법대출과 불법 탈세를 살폈다.',
         '합수본은 국회의원의 부동산 관련 뇌물수수, 차명거래 혐의, LH 및 서울주택도시공사(SH) 직원들의 뇌물수수 혐의에 대해서도 수사를 진행 중이다. 특히 LH의 경우 경찰은 직원 77명과 친인척·지인 74명등 151명을 적발해 현재까지 4명을 구속하고 126명에 대해 계속 수사를 진행하고 있다.',
         '해당 영상은 코로나19가 발생하기 전인 지난 2017년 12월 29일 중국중앙방송(CCTV)을 통해 공개된 영상이다. 이 영상에는 박쥐한테 물린 자국이 심하게 부풀어 오른 사진도 포함됐다. 영상에 등장한 연구진은 박쥐가 자신의 장갑을 뚫고 물었다며 “바늘에 찔린 기분”이라고 설명했다.',
         "이용구 법무부 차관이 차관에 임명되기 한 달 전쯤인 지난해 11월, 술에 취해 택시기사를 폭행하는 장면이 담긴 영상을 SBS가 단독으로 입수했습니다. 37초짜리 이 영상에는 이용구 차관이 택시기사의 목을 조르고 또 욕을 하는 모습이 고스란히 담겨 있습니다. 이용구 차관은 지난주 사표를 냈지만 아직 수리되지 않아서 지금도 '법무부 차관' 신분입니다. 그럼 먼저 지난해 11월 6일 밤, 이용구 차관이 타고 있던 택시 안에서 무슨 일이 있었는지부터 함께 보시겠습니다.",
         '이씨는 지난해 7월 11일 성남시의 한 가정집에서 전 여자친구 A씨를 흉기로 여러 차례 찔러 살해한 혐의로 기소됐다. A씨의 승용차와 신용카드를 훔쳐 달아난 혐의도 받고 있다. 이씨는 범행 후 A씨의 차를 몰고 전남 고흥으로 도주해 극단적 선택을 시도했지만 출동한 경찰에 체포돼 목숨을 건졌다.',
         ]

In [ ]:
qurry = ['불이 난 건물과 이웃한 건물은 지은 지 오래돼 스프링클러나 화재 경보 장치가 없던 상황.30m 떨어진 곳에는 주유소까지 있어서 불이 번져 불티라도 튀었으면 더 큰 참사가 발생할 수도 있었습니다.이 때문에 주변 상인들은 김 씨가 큰불을 막았다며 은인이라며 고맙다고 말합니다.[주변 상인 : 고맙다고 내가 인사를 해야 하는데, 너무 고맙습니다. 그때 그렇게 해주셨으니까 빨리 불이 꺼졌었지.]화재 초기에 소화기 1대의 위력이 얼마나 강력한지 몸소 보여준 김 씨.정작 김 씨는 코로나19 때문에 힘든 상인들을 걱정하는 마음에 당연히 한 일이라며 겸손해했습니다.[김상남 / 화재 진압 버스 기사 : 건물마저 타면 그분들이 진짜 큰일 나겠구나 하는 마음을 갖고 달려든 것 같습니다.]',
         "'대구 키다리 아저씨'가 올해도 이름을 숨긴 채 5천여만 원을 기부했습니다.대구사회복지공동모금회는 10년 가까이 익명으로 기부해온 이른바 '키다리 아저씨'가 올해도 5천여만 원을 기부했다고 밝혔습니다.키다리 아저씨 부부는 저녁 식사를 함께하자며 모금회 직원을 초대했고, 이 자리에서 5천4만 원짜리 수표와 메모지가 든 봉투를 건넸습니다.",
         '전주시는 얼굴없는 천사의 뜻을 기리고 아름다운 기부문화가 확산하도록 노송동주민센터 화단에 “당신은 어둠 속의 촛불처럼 세상을 밝고 아름답게 만드는 참사람입니다. 사랑합니다”라는 글귀가 새겨진 ‘얼굴없는 천사의 비’를 2009년 12월 세웠다. 주민들은 얼굴없는 천사의 선행을 본받자는 뜻에서 숫자 천사(1004)를 본따서 10월4일을 ‘천사의 날’로 정하고 불우이웃을 돕고 있다. ',
         ]

In [ ]:
tokenized_sentences = tokenizer(
    qurry,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=50,
    add_special_tokens=True,
    return_token_type_ids = True
    )

In [ ]:
output = model(input_ids=tokenized_sentences['input_ids'].to(device),
      attention_mask=tokenized_sentences['attention_mask'].to(device),
      token_type_ids=tokenized_sentences['token_type_ids'].to(device),
      labels=None,
      )

In [ ]:
torch.argmax(output[0], 1)

tensor([0, 1, 1], device='cuda:0')

In [ ]:
torch.argmax(output[0], 1)

tensor([0, 1, 0, 1, 0], device='cuda:0')